# LangChain Agents: Complete Guide

This notebook demonstrates three different approaches to creating LangChain agents:
1. **Basic Agent with Model String** - Simplest approach for quick prototyping
2. **Agent with Explicit Model Instance** - Full control over model configuration
3. **Dynamic Model Selection** - Cost optimization with automatic model switching

## Prerequisites

Make sure you have the required packages installed and Ollama running:

```bash
pip install -U langchain langchain-community langchain-core langgraph
pip install -U ddgs python-dotenv
```

Local LLM Serving with Ollama:
```bash
ollama pull qwen3
ollama pull gpt-oss
ollama serve
```

---
# Part 1: Basic Agent with Model String

The simplest way to create a LangChain agent using a model string.

In [ ]:
# Import required modules
from langchain.agents import create_agent
import tools

In [ ]:
# Test the web search tool
print(tools.web_search.invoke("python programming"))

In [ ]:
# Create agent with model string - simplest approach
agent = create_agent(
    "ollama:qwen3",
    tools=[tools.web_search]
)

In [ ]:
# Test the agent with a search query
result = agent.invoke({
    "messages": "Search for python tutorials"
})

print(result)

---
# Part 2: Agent with Explicit Model Instance

Full control over model configuration by creating an explicit ChatOllama instance.

## Model Parameters Explained
- **temperature**: Controls randomness (0.0 = deterministic, 1.0 = very creative)
- **num_predict**: Maximum tokens to generate (similar to max_tokens in OpenAI)
- **top_k**: Number of highest probability tokens to consider
- **top_p**: Cumulative probability threshold for token selection
- **repeat_penalty**: Penalty for repeating tokens
- **num_ctx**: Context window size

In [ ]:
# Import required modules
from langchain_ollama import ChatOllama
from langchain.agents import create_agent
import tools

In [ ]:
# Test the calculate tool
tools.calculate.invoke("3*4+2-10/20")

In [ ]:
# Create model instance with custom parameters
model = ChatOllama(
    model="qwen3",
    base_url="http://localhost:11434",
    temperature=0
)

In [ ]:
# Create agent with explicit model instance
agent = create_agent(model, tools=[tools.calculate])

print("Agent created with explicit model instance!")
print("Full control over temperature, token limits, and other parameters")

In [ ]:
# Test the agent with a mathematical calculation
result = agent.invoke({
    "messages": "What's 15 * 27 + 100?"
})

print(f"Response: {result['messages'][-1].content}")

In [ ]:
# View full result
result

In [ ]:
# Pretty print the last message
result['messages'][-1].pretty_print()

## Experimenting with Different Settings

Let's compare different model configurations to see how they affect the output:

In [ ]:
question = "Explain what 2 + 2 equals and show your reasoning"

# Configuration 1: Conservative/Deterministic
print("=== Configuration 1: Conservative/Deterministic ===")
llm1 = ChatOllama(
    model="qwen3",
    temperature=0,
    top_p=1,
    repeat_penalty=1.1,
    num_predict=500,
    num_ctx=4096,
    reasoning=True
)
agent = create_agent(llm1, tools=[tools.calculate])
result = agent.invoke({"messages": question})
print(f"Conservative output: {result['messages'][-1].content}")
print()

In [ ]:
print("=== Configuration 2: Balanced/Production ===")
llm = ChatOllama(
    model="qwen3",
    temperature=2,
    top_k=2000,
    repeat_penalty=1.15,
    repeat_last_n=64,
    num_predict=1000,
    num_ctx=8192,
    keep_alive="5m",
    reasoning=True
)

agent = create_agent(llm, tools=[tools.calculate])
result = agent.invoke({"messages": question})
print(result['messages'][-1].content)
print()

# Part 3: Dynamic Model Selection (Qwen3 → GPT-OSS)

Cost-optimization strategy where the agent automatically switches between models based on conversation complexity.

## Selection Logic
- **< 10 messages**: Use Qwen3 (fast, efficient)
- **≥ 10 messages**: Use GPT-OSS (better reasoning, longer context)

## Real-World Applications
- Customer service bots (simple queries → Qwen3, complex issues → GPT-OSS)
- Research assistants (quick facts → Qwen3, analysis → GPT-OSS)

In [1]:
# Import required modules
from langchain_ollama import ChatOllama
from langchain.agents import create_agent, AgentState
from langgraph.runtime import Runtime
import tools

## Model Selection Function

This function automatically chooses between Qwen3 and GPT-OSS based on conversation length:

In [ ]:
# Define tool list for both models
tool_list = [tools.web_search, tools.calculate]

def select_model(state: AgentState, runtime: Runtime) -> ChatOllama:
    """Choose between Qwen3 and GPT-OSS based on conversation length."""
    messages = state["messages"]
    message_count = len(messages)
    
    if message_count < 10:
        print(f"Using Qwen3 for {message_count} messages")
        return ChatOllama(model="qwen3", temperature=0.1).bind_tools(tool_list)
    else:
        print(f"Switching to GPT-OSS for {message_count} messages")
        return ChatOllama(model="gpt-oss", temperature=0.0, num_predict=2000).bind_tools(tool_list)

print("Model selection function defined!")
print("Logic: < 10 messages = Qwen3, >= 10 messages = GPT-OSS")

## Creating the Dynamic Agent

Create an agent that uses our dynamic model selection function:

In [ ]:
# Create agent with dynamic model selection
agent = create_agent(select_model, tools=tool_list)

In [ ]:
def demo_conversation_progression(messages):
    """Demonstrate how the agent switches models as conversation grows."""
        
    # Create a mock state to test model selection
    mock_state = {"messages": messages}
    
    agent = create_agent(select_model, tools=tool_list)
    result = agent.invoke(mock_state)

    return result

messages = [
        "Hello", "How are you?", "What's the weather?", "My name is Laxmi Kant Tiwari",
        "Explain machine learning", "What about deep learning?", "Show me examples",
        "How does this work?", "Give me more details"]

print(f"Len of messages: {len(messages)}")

result = demo_conversation_progression(messages)

In [ ]:
messages = [
        "Hello", "How are you?", "What's the weather?", "My name is Laxmi Kant Tiwari",
        "Explain machine learning", "What about deep learning?", "Show me examples",
        "How does this work?", "Give me more details", "What is my name?"]

print(f"Len of messages: {len(messages)}")

result = demo_conversation_progression(messages)